In [1]:
# Run this cell to import the neccessary imports.
# Make sure you have the following files in your working directory
#        "inputs.txt"
#        "relu_bias.txt"
#        "relu_weights_gru_basic.txt"
#        "sigmoid_bias.txt"
#        "sigmoid_weights_gru_basic.txt"

import numpy as np
import os

# # input binaries
# in_bin_folder = './input_binaries'
# if not os.path.exists(in_bin_folder):
#     os.makedirs(in_bin_folder)

# # relu package sv file folder
# relu_pkg_folder = './relu_weights_biases_pkgs'
# if not os.path.exists(relu_pkg_folder):
#     os.makedirs(relu_pkg_folder)

# # sigmoid package sv file folder
# sig_pkg_folder = './sigmoid_weights_biases_pkgs'
# if not os.path.exists(sig_pkg_folder):
#     os.makedirs(sig_pkg_folder)


# # gru package sv file folder
gru_pkg_folder = './gru_basic_weights/gru_weights_biases_pkgs'
if not os.path.exists(gru_pkg_folder):
    os.makedirs(gru_pkg_folder)

# # binary that goes to tanh bram/look up table
if not os.path.exists('./binaries/tanh_BRAM_binaries'):
    os.makedirs('./tanh_BRAM_binaries')

# # dense 1 package sv file folder
dense_1_pkg_folder = './weights/dense_1_weights_biases_pkgs'
if not os.path.exists(dense_1_pkg_folder):
    os.makedirs(dense_1_pkg_folder)

def flip_bits(val, bits):
    return ((val ^ (2**bits - 1)) + 1)

def sigmoid(x):
    return 1/(1+(np.e**(-x)))

# really innefficient but calling "sed" subprocess didn't work
# Removes negative sign "-" from file
def remove_negs(tmp_file, dest_file_name):
    output_file = open(tmp_file, 'r')
    dest_file = open(dest_file_name, 'w')
    file = output_file.read()
    file = file.replace('b-', 'b')
    dest_file.write(file)
    # close files
    dest_file.close()
    output_file.close()
    
# =========================================================================
# pkg functions (for sub-processes)
# =========================================================================
def get_pkg_name(name, width, nfrac):
    return str(name) + '_' + str(width) + '_' + str(width - nfrac)

def conv_to_str(num, width, nfrac):

    # shift the decimal point to the right
    div = (2**nfrac)*1.0
    x = round(float(num * div // 1))

    if (x < 0):
        return format(flip_bits(x, width), '0'+ str(width) + 'b')
    return format(x, '0'+ str(width) + 'b')


# =========================================================================

# print(conv_to_str(4.127311706542969, 4, 2))
print(conv_to_str(-1.5, 4, 2))
# print(conv_to_str(-2.344774007797241, 10, 5))



-1010


In [2]:
from tensorflow.keras.models import load_model

model = load_model('../../../../documents/Benchmarks/RNN/ftag/gru_model/gru_basic.h5')
model.summary()

2024-11-08 15:17:49.331279: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-08 15:17:50.553404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-08 15:17:50.566563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-08 15:17:50.566629: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/000

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 120)               46080     
                                                                 
 dense_1 (Dense)             (None, 3)                 363       
                                                                 
Total params: 46,443
Trainable params: 46,443
Non-trainable params: 0
_________________________________________________________________


0:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-08 15:17:50.568756: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-08 15:17:50.568810: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-08 15:17:50.660207: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-08 15:17:50.660319: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been bu

In [3]:
# Extract weights and biases for each layer (gru layer separated to 3 dense) and print the weights to output and to files with name layer_weight.txt and layer_bias.txt
for layer in model.layers:
    print(layer.name)
    if layer.name == 'gru_1':

        print(layer.get_config())

        weights = layer.get_weights()
        print("Number of weights tensors: " + str(len(weights)))

        # W for x, U for h, B for biases
        W = weights[0]
        U = weights[1]
        B = weights[2]

        print("input size " + str(W.shape[0]))
        print("hidden state size " + str(layer.units))

        # extract three weights and biases for the three dense layers

        reset_gate_weight = np.concatenate((W[:, :layer.units], U[:, :layer.units]), axis = 0)
        update_gate_weight = np.concatenate((W[:, layer.units:2*layer.units], U[:, layer.units:2*layer.units]), axis=0)
        candidate_gate_weight = np.concatenate((W[:, 2*layer.units:], U[:, 2*layer.units:]), axis=0)

        # combine input biases and recurrent biases
        # notice recurrent biases is used when reset_after config set to True for the gru layer (default is True)
        reset_gate_bias = np.sum(B[:, :layer.units], axis = 0)
        update_gate_bias = np.sum(B[:, layer.units:2*layer.units], axis = 0)
        candidate_gate_bias = np.sum(B[:, 2*layer.units:], axis = 0)

        print("reset_gate_weight")
        print(reset_gate_weight.shape)
        print("update_gate_weight")
        print(update_gate_weight.shape)
        print("candidate_gate_weight")
        print(candidate_gate_weight.shape)

        print("reset_gate_bias")
        print(reset_gate_bias.shape)
        print("update_gate_bias")
        print(update_gate_bias.shape)
        print("candidate_gate_bias")
        print(candidate_gate_bias.shape)


        # find maximum and minimum values for each weight and bias
        print("max and min values")
        print("reset_gate_weight")
        print(np.max(reset_gate_weight))
        print(np.min(reset_gate_weight))
        print("update_gate_weight")
        print(np.max(update_gate_weight))
        print(np.min(update_gate_weight))
        print("candidate_gate_weight")
        print(np.max(candidate_gate_weight))
        print(np.min(candidate_gate_weight))

        print("reset_gate_bias")
        print(np.max(reset_gate_bias))
        print(np.min(reset_gate_bias))
        print("update_gate_bias")
        print(np.max(update_gate_bias))
        print(np.min(update_gate_bias))
        print("candidate_gate_bias")
        print(np.max(candidate_gate_bias))
        print(np.min(candidate_gate_bias))
        

        # write to npy files
        np.savetxt('reset_gate_weights_gru_basic.txt', reset_gate_weight.flatten())
        np.savetxt('update_gate_weights_gru_basic.txt', update_gate_weight.flatten())
        np.savetxt('candidate_gate_weights_gru_basic.txt', candidate_gate_weight.flatten())

        np.savetxt('reset_gate_biases_gru_basic.txt', reset_gate_bias.flatten())
        np.savetxt('update_gate_biases_gru_basic.txt', update_gate_bias.flatten())
        np.savetxt('candidate_gate_biases_gru_basic.txt', candidate_gate_bias.flatten())


    elif layer.name != "input_1":
        # all other layers are dense with relu activation
        
        # print layer info (activation)
        print(layer.get_config())
        dense_weights = layer.get_weights()
        weights = dense_weights[0]
        biases = dense_weights[1]

        # print weights and biases
        print("weights shape")
        print(weights.shape)
        print("biases shape")
        print(biases.shape)

        # find maximum and minimum values for each weight and bias
        print("max and min values")
        print("weights")
        print(np.max(weights))
        print(np.min(weights))
        print("biases")
        print(np.max(biases))
        print(np.min(biases))    

        # # write to a txt file, flatten the weights
        np.savetxt(layer.name + '_weights_gru_basic.txt', weights.flatten())
        np.savetxt(layer.name + '_biases_gru_basic.txt', biases.flatten())
    
    print("=====================================================================================================")


# ALter this to reflect state of files
NUM_INPUTS = 10

INPUT_SIZE_DENSE_0 = 120
OUTPUT_SIZE_DENSE_0 = 50

INPUT_SIZE_DENSE_1 = 120
OUTPUT_SIZE_DENSE_1 = 3

INPUT_SIZE_OUT_DENSE = 10
OUTPUT_SIZE_OUT_DENSE = 3

INPUT_SIZE_GRU = 6
OUTPUT_SIZE_GRU = 120


gru_1
{'name': 'gru_1', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 2, 6), 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 120, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.0, 'recurrent_dropout': 0.0, 'implementation': 2, 'reset_after': True}
Number of weights tensors: 3
input size 6
hidden state size 120
reset_gate_weight
(126, 120)
update_gate_weight
(126, 120)
candidate_gate_weight
(126, 120)
reset_gate_b

In [4]:
# =======================================================
# generate dense wights and bias package
# =======================================================

def generate_dense_pkg_file(name, dest_folder, width, nfrac, weights_file, biases_file, input_size, output_size):
# arguments:
# name: name of the layer
# width: width of the binary
# nfrac: number of fractional bits
# weights_file: file containing weights
# biases_file: file containing biases
# num_weights: number of weights
# num_biases: number of biases
    
    # determine the maximum and minimum values the fix point number can represent
    max_val = (2**(width-1) - 1)/(2**nfrac)
    min_val = -2**(width-1)/(2**nfrac)
    print("Max value: " + str(max_val))
    print("Min value: " + str(min_val))

    # open source files
    src_weights_file = open(weights_file, 'r')
    src_biases_file = open(biases_file, 'r')
    
    package_name = get_pkg_name(name, width, nfrac)

    # make the final output files
    file_name = dest_folder + package_name + ".sv"
    dest_file = open(file_name, 'w')

    print(package_name)

    # first decalaration lines
    dest_file.write("// Width: " + str(width) + "\n// NFRAC: " + str(nfrac) + "\n")
    dest_file.write("package " + str(package_name) + ';\n\n')
    dest_file.write(f"localparam logic signed [{str(width-1)}:0] weights [{str(input_size)}][{str(output_size)}] = \'{{ \n")

    # convert weights (from floats) into binary
    for i in range(input_size):
        dest_file.write("{")
        for j in range(output_size):
            flt_num = float(src_weights_file.readline())
                        
            if (flt_num > max_val):
                flt_num = max_val
            elif (flt_num < min_val):
                flt_num = min_val

            binary_num = conv_to_str(flt_num, width, nfrac).replace('-', '')

            # Overflow checking, now handled by converting extreme values to max/min values
            if (len(binary_num) < width):
                print("ERROR: binary number too large")
                print("binary number: " + binary_num)
                print("float number: " + str(flt_num))
                print("width: " + str(width))
                print("binary number length: " + str(len(binary_num)))
                print("from file: " + weights_file)
                raise ValueError("Binary number width inconsistent")


            dest_file.write(str(width) + "\'b" + binary_num)
            if (j != output_size - 1):
                dest_file.write(", ")
            else:
                dest_file.write("}")

        if (i != (input_size-1)): # no comma for last term
            # OPTIONAL: adds floating number as comment as reference
            dest_file.write(", " + "\n")
        else:
            dest_file.write("\n")                
    
    # switch to declaring biases
    dest_file.write("};\n\n")
    dest_file.write("localparam logic signed ["+ str(width-1) + ":0] bias [" + str(output_size) + "] = \'{\n")
    
    # convert bias (from floats) into binary
    for i in range(output_size):
        flt_num = float(src_biases_file.readline())
        
        # e.g. write line "17'b01011011011111001,"
        dest_file.write(str(width) + "\'b" + conv_to_str(flt_num, width, nfrac).replace('-', ''))
        
        if (i != (output_size-1)): # no comma for last term
            # OPTIONAL: adds floating number as comment as reference
            dest_file.write(",  // " + str(flt_num) + "\n")
        else:
            dest_file.write("   // " + str(flt_num) + "\n")
        
        
    # finish off declarations
    dest_file.write("};\nendpackage")
    
    
    dest_file.close()
    src_biases_file.close()
    src_weights_file.close()
    

In [5]:
def generate_gru_pkg_file(width, nfrac, input_size, output_size):

    # input size for each gru internal gate is input_size + h, which is input_size + output_size
    generate_dense_pkg_file("candidate_gate", "./gru_basic_weights/gru_weights_biases_pkgs/candidate_gate_weights_biases_pkgs/", width, nfrac, "candidate_gate_weights_gru_basic.txt", "candidate_gate_biases_gru_basic.txt", input_size + output_size, output_size)
    generate_dense_pkg_file("reset_gate", "./gru_basic_weights/gru_weights_biases_pkgs/reset_gate_weights_biases_pkgs/", width, nfrac, "reset_gate_weights_gru_basic.txt", "reset_gate_biases_gru_basic.txt", input_size + output_size, output_size)
    generate_dense_pkg_file("update_gate", "./gru_basic_weights/gru_weights_biases_pkgs/update_gate_weights_biases_pkgs/", width, nfrac, "update_gate_weights_gru_basic.txt", "update_gate_biases_gru_basic.txt", input_size + output_size, output_size)


In [6]:
# # =======================================================
# # generate input vals
# # =======================================================

# def generate_input_bin_file(width, nfrac):
#     div = (2**nfrac)*1.0

#     src_file = open("inputs.txt", 'r')
    
#     file_name = in_bin_folder + "/bw" + str(width) + "_nfrac" + str(nfrac) + "_inputs.mem"
#     tmp_file = in_bin_folder + "/tmp"
#     dest_file = open(tmp_file, 'w')


#     for i in range(NUM_INPUTS):
#         x = float(src_file.readline())
#         dest_file.write(conv_to_str(x, width, nfrac))
#         dest_file.write("\n")
    
#     #subprocess.call(["sed", 's/-//g', file_name]) # doesn't work
#     dest_file.close()
#     src_file.close()
    
#     # replace negative '-' character
#     remove_negs(tmp_file, file_name)

# # =+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=

# # Set width and fractional bits you want to generate
# width = 10
# nfrac = 5

# # run function
# # generate_input_bin_file(width, nfrac)


In [7]:
# Weight generating file specific to ftag model.

# # Set width and fractional bits you want to generate
# width = 10
# nfrac = 5

# # run function
# generate_dense_pkg_file("dense_0", width, nfrac, "dense_0_weights_gru_basic.txt", "dense_0_biases_gru_basic.txt", NUM_WEIGHTS_DENSE_0, NUM_BIAS_DENSE_0)
# generate_dense_pkg_file("dense_1", width, nfrac, "dense_1_weights_gru_basic.txt", "dense_1_biases_gru_basic.txt", NUM_WEIGHTS_DENSE_1, NUM_BIAS_DENSE_1)

# generate set of files for bit width 4-32
for i in range(4,33):
    width = i
    nfrac = (i+1) // 2
    
    # generate_input_bin_file(width, nfrac)
    # generate_dense_pkg_file("dense_0", "./gru_basic_weights/dense_0_weights_biases_pkgs/", width, nfrac, "dense_0_weights_gru_basic.txt", "dense_0_biases_gru_basic.txt", INPUT_SIZE_DENSE_0, OUTPUT_SIZE_DENSE_0)
    generate_dense_pkg_file("dense_1", "./gru_basic_weights/dense_1_weights_biases_pkgs/", width, nfrac, "dense_1_weights_gru_basic.txt", "dense_1_biases_gru_basic.txt", INPUT_SIZE_DENSE_1, OUTPUT_SIZE_DENSE_1)
    generate_gru_pkg_file(width, nfrac, INPUT_SIZE_GRU, OUTPUT_SIZE_GRU)
    # output layer
    # generate_dense_pkg_file("output", "./gru_basic_weights/output_weights_biases_pkgs/", width, nfrac, "output_softmax_weights_gru_basic.txt", "output_softmax_biases_gru_basic.txt", INPUT_SIZE_OUT_DENSE, OUTPUT_SIZE_OUT_DENSE)

# width = 16
# nfrac = 10
# generate_gru_pkg_file(width, nfrac, INPUT_SIZE_GRU, OUTPUT_SIZE_GRU)
# generate_dense_pkg_file("dense_0", "./weights/dense_0_weights_biases_pkgs/", width, nfrac, "dense_0_weights_gru_basic.txt", "dense_0_biases_gru_basic.txt", INPUT_SIZE_DENSE_0, OUTPUT_SIZE_DENSE_0)
# generate_dense_pkg_file("dense_1", "./weights/dense_1_weights_biases_pkgs/", width, nfrac, "dense_1_weights_gru_basic.txt", "dense_1_biases_gru_basic.txt", INPUT_SIZE_DENSE_1, OUTPUT_SIZE_DENSE_1)
# # output layer
# generate_dense_pkg_file("output", "./weights/output_weights_biases_pkgs/", width, nfrac, "output_softmax_weights_gru_basic.txt", "output_softmax_biases_gru_basic.txt", INPUT_SIZE_OUT_DENSE, OUTPUT_SIZE_OUT_DENSE)

Max value: 1.75
Min value: -2.0
dense_1_4_2
Max value: 1.75
Min value: -2.0
candidate_gate_4_2
Max value: 1.75
Min value: -2.0
reset_gate_4_2
Max value: 1.75
Min value: -2.0
update_gate_4_2


Max value: 1.875
Min value: -2.0
dense_1_5_2
Max value: 1.875
Min value: -2.0
candidate_gate_5_2
Max value: 1.875
Min value: -2.0
reset_gate_5_2
Max value: 1.875
Min value: -2.0
update_gate_5_2
Max value: 3.875
Min value: -4.0
dense_1_6_3
Max value: 3.875
Min value: -4.0
candidate_gate_6_3
Max value: 3.875
Min value: -4.0
reset_gate_6_3
Max value: 3.875
Min value: -4.0
update_gate_6_3
Max value: 3.9375
Min value: -4.0
dense_1_7_3
Max value: 3.9375
Min value: -4.0
candidate_gate_7_3
Max value: 3.9375
Min value: -4.0
reset_gate_7_3
Max value: 3.9375
Min value: -4.0
update_gate_7_3
Max value: 7.9375
Min value: -8.0
dense_1_8_4
Max value: 7.9375
Min value: -8.0
candidate_gate_8_4
Max value: 7.9375
Min value: -8.0
reset_gate_8_4
Max value: 7.9375
Min value: -8.0
update_gate_8_4
Max value: 7.96875
Min value: -8.0
dense_1_9_4
Max value: 7.96875
Min value: -8.0
candidate_gate_9_4
Max value: 7.96875
Min value: -8.0
reset_gate_9_4
Max value: 7.96875
Min value: -8.0
update_gate_9_4
Max value: 15.

In [8]:
# # =======================================================
# # generate sigmoid wights and bias package
# # =======================================================


# def generate_sigmoid_pkg_file(width, nfrac):
#     # open source files
#     src_weights_file = open("sigmoid_weights_gru_basic.txt", 'r')
#     src_biases_file = open("sigmoid_bias.txt", 'r')
    
#     # make the final output files
#     file_name = sig_pkg_folder + "/" + get_pkg_name("sigmoid", width, nfrac) + ".sv"
#     tmp_file = sig_pkg_folder + "/tmp"
#     dest_file = open(tmp_file, 'w') # write to tmp file, will write to final when cleaning up '-' signs

#     # first decalaration lines
#     dest_file.write("// Width: " + str(width) + "\n// NFRAC: " + str(nfrac) + "\n")
#     dest_file.write("package " + str(get_pkg_name("sigmoid", width, nfrac)) + ';\n\n')
#     dest_file.write("localparam logic signed ["+ str(width-1) + ":0] weights [" + str(NUM_WEIGHTS_SIGMOID) + "] = \'{\n")

#     # convert weights (from floats) into binary
#     for i in range(NUM_WEIGHTS_SIGMOID):
#         flt_num = float(src_weights_file.readline())
        
#         # e.g. write line "17'b01011011011111001,"
#         dest_file.write(str(width) + "\'b" + conv_to_str(flt_num, width, nfrac))
        
#         if (i != (NUM_WEIGHTS_SIGMOID-1)): # no comma for last term
#             # OPTIONAL: adds floating number as comment as reference
#             dest_file.write(",  // " + str(flt_num) + "\n")
#         else:
#             dest_file.write("   // " + str(flt_num) + "\n")
    
#     # switch to declaring biases
#     dest_file.write("};\n\n")
#     dest_file.write("localparam logic signed ["+ str(width-1) + ":0] bias [" + str(NUM_BIAS_SIGMOID) + "] = \'{\n")
    
#     # convert bias (from floats) into binary
#     for i in range(NUM_BIAS_SIGMOID):
#         flt_num = float(src_biases_file.readline())
        
#         # e.g. write line "17'b01011011011111001,"
#         dest_file.write(str(width) + "\'b" + conv_to_str(flt_num, width, nfrac))
        
        
#         if (i != (NUM_BIAS_SIGMOID-1)): # no comma for last term
#             # OPTIONAL: adds floating number as comment as reference
#             dest_file.write(",  // " + str(flt_num) + "\n")
#         else:
#             dest_file.write("   // " + str(flt_num) + "\n")
    
#     # finish off declarations
#     dest_file.write("};\nendpackage")
    
    
    
#     #subprocess.call(["sed", 's/-//g', file_name]) # doesn't work
#     dest_file.close()
#     src_biases_file.close()
#     src_weights_file.close()
    
#     # replace negative '-' character
#     remove_negs(tmp_file, file_name)


# # =+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=

# # Set width and fractional bits you want to generate
# width = 10
# nfrac = 5

# # run function
# # generate_sigmoid_pkg_file(width, nfrac)

In [9]:
# =======================================================
# generate sigmoid bram
# =======================================================


def generate_sigmoid_bram_bin_file(width, nfrac, table_size_pow, prec):
    file = open("./sigmoid_BRAM_vals.txt", 'w')
    
    # number of table entries
    N_TABLE = 2**table_size_pow

    index = np.arange(-8, 8, 16.0/N_TABLE)

    for n in index:
        file.write(str(sigmoid(n)))
        file.write("\n")

    file.close()

    # =================================================================
    # now convert into binary
    div = (2**prec)*1.0

    src_file = open("sigmoid_BRAM_vals.txt", 'r')
    
    file_name = "./binaries/sigmoid_BRAM_binaries" + "/memw" + str(prec) + "_size" + str(int(N_TABLE)) + "_sigmoidBRAM.mem"
    tmp_file = "./binaries/sigmoid_BRAM_binaries" + "/tmp.mem"
    dest_file = open(tmp_file, 'w')

    for i in range(int(N_TABLE)):
        x = round(float(src_file.readline())*div // 1)
    
        if (x < 0):
            dest_file.write(format(flip_bits(x, width), '0'+ str(prec) + 'b'))
            dest_file.write("\n")
        else:
            dest_file.write(format(x, '0'+ str(prec) + 'b'))
            dest_file.write("\n")
        
    dest_file.close()
    src_file.close()
    
    # replace negative '-' character
    remove_negs(tmp_file, file_name)


# =+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=

# Set width and fractional bits you want to generate
width = 10
nfrac = 5

# run function
generate_sigmoid_bram_bin_file(width, nfrac, 10, 10)



In [10]:
# =======================================================
# generate tanh bram (input range from 0 to 8, since tanh is symmetric around 0)
# =======================================================
def generate_tanh_bram_bin_file(width, nfrac, table_size_pow, prec):
    file = open("./tanh_BRAM_vals.txt", 'w')
    
    # number of table entries
    N_TABLE = 2**table_size_pow

    # initialize array with zeros
    index = np.arange(0, 8, 8.0/N_TABLE)

    for n in index:
        file.write(str(np.tanh(n)))
        file.write("\n")

    file.close()

    # =================================================================
    # now convert into binary
    div = (2**prec)*1.0

    src_file = open("tanh_BRAM_vals.txt", 'r')
    
    file_name = "./binaries/tanh_BRAM_binaries" + "/memw" + str(prec) + "_size" + str(int(N_TABLE)) + "_tanhBRAM.mem"
    dest_file = open(file_name, 'w')

    for i in range(int(N_TABLE)):
        x = round(float(src_file.readline())*div // 1)
    
        if (x < 0):
            dest_file.write(format(flip_bits(x, width), '0'+ str(prec) + 'b'))
            dest_file.write("\n")
        else:
            dest_file.write(format(x, '0'+ str(prec) + 'b'))
            dest_file.write("\n")
        
    dest_file.close()
    src_file.close()

generate_tanh_bram_bin_file(17, 10, 9, 10)


In [11]:
# ===============================================================================================
# ===============================================================================================
# ===============================================================================================
# ===============================================================================================

In [12]:

# # generate specific set or write a simple for loop to generate multiple sets of files

# width = 17              # width of numbers
# nfrac = 10              # number of fractional bits
# table_size_pow = 10     # BRAM size (2**table_size_pow)
# prec = 10               # BRAM precision

# # generate_input_bin_file(width, nfrac)
# # generate_relu_pkg_file(width, nfrac)
# # generate_sigmoid_pkg_file(width, nfrac)
# # generate_sigmoid_bram_bin_file(width, nfrac, table_size_pow, prec)

# # generate set of files for bit width 4-32
# for i in range(4,33):
#     width = i
#     nfrac = (i+1) // 2
    
#     # generate_input_bin_file(width, nfrac)
#     generate_relu_pkg_file(width, nfrac)
#     generate_sigmoid_pkg_file(width, nfrac)
    
# generate_sigmoid_bram_bin_file(width, nfrac, table_size_pow, prec)

